<a href="https://colab.research.google.com/github/sarjakpatel/SJSU/blob/main/CMPE%20256/Assignment%2011/CMPE_256_Market_Basket_Item_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CMPE 256 Market Basket Item Assignment

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install apyori

In [3]:
from apyori import apriori, load_transactions

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/Market Basket Item Dataset/TRAIN-ARULES.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/Market Basket Item Dataset/testarules.csv", keep_default_na = False).values

In [6]:
train_data.product_name = ["".join(i.split(",")) for i in train_data.product_name]

In [7]:
train_data.head()

,order_id,user_id,product_name
0,1483,90,Organic Pink Lemonade Bunny Fruit Snacks
1,1483,90,Dark Chocolate Minis
2,1483,90,Sparkling Water Natural Mango Essenced
3,1483,90,Peach-Pear Sparkling Water
4,1483,90,Organic Heritage Flakes Cereal


In [8]:
test_number_items = [len(i) for i in test_data]
test_data = [[" ".join(i.split(",")) for i in j if i != ""] for j in test_data]

In [9]:
test_data

[['Dark Chocolate Minis',
  'Organic Pink Lemonade Bunny Fruit Snacks',
  'Peach-Pear Sparkling Water']]

In [10]:
train_data[train_data.order_id == 1483].shape

(12, 3)

In [11]:
train_data[train_data.user_id == 90].shape

(670, 3)

From above 2 cell results, We can infer that single user has multiple orders.

In [12]:
train_order_labels = train_data.order_id.unique()
transactions = [train_data[train_data.order_id == i]["product_name"].tolist() for i in train_order_labels]

In [13]:
minimum_support = 0.0045
apriori_train = apriori(transactions = transactions, min_support = minimum_support)
apriori_train_result = list(apriori_train)
apriori_train_result = pd.DataFrame(apriori_train_result)
apriori_train_result["confidence"] = 0.0
apriori_train_result["lift"] = 0.0

In [14]:
for i in range(apriori_train_result.shape[0]):
  apriori_train_result["confidence"][i] = apriori_train_result.ordered_statistics[i][0].confidence
  apriori_train_result["lift"][i] = apriori_train_result.ordered_statistics[i][0].lift
  apriori_train_result["items"][i] = tuple(apriori_train_result["items"][i])
apriori_train_result = apriori_train_result.drop("ordered_statistics", axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [15]:
apriori_train_result.head()

,items,support,confidence,lift
0,"(0% Greek Strained Yogurt,)",0.009873,0.009873,1.0
1,"(100% Juice Variety Pack,)",0.004937,0.004937,1.0
2,(100% Premium Select Not From Concentrate Pure...,0.009168,0.009168,1.0
3,"(100% Recycled Paper Towels,)",0.005642,0.005642,1.0
4,"(1500 Pale Ale,)",0.014810,0.014810,1.0


In [16]:
def predict(dataset, test, num_items):
  length = np.array([len(i) for i in dataset["items"]])
  itemSet = set()
  for item in dataset[length == num_items]["items"]:
    if all([True if i in item else False for i in test]) :
      itemSet.add(item)
  return itemSet

In [17]:
result = {}
for i, (test_row, test_number_row) in enumerate(zip(test_data, test_number_items)):
  result["row " + str(i)] = predict(apriori_train_result, test_row, test_number_row)

In [18]:
for i,item in enumerate(result.keys()):
  print("Basket items prediction for transaction:-")
  print("\n Transaction",i+1,": { "," ,".join(test_data[i]),"}\n")
  for j,prediction in enumerate(result[item]):
    print(str(j+1)+". {"+", ".join(set(prediction).difference(test_data[i])),"}")

Basket items prediction for transaction:-

 Transaction 1 : {  Dark Chocolate Minis ,Organic Pink Lemonade Bunny Fruit Snacks ,Peach-Pear Sparkling Water }

1. {Organic Heritage Flakes Cereal, Maple Pumpkin Seeds with Sea Salt Chewy with a Crunch Granola Bars }
2. {Organic Heritage Flakes Cereal, Sparkling Water Natural Mango Essenced }
3. {Organic Heritage Flakes Cereal, Organic Graham Crunch Cereal }
4. {Organic Graham Crunch Cereal, Sparkling Water Natural Mango Essenced }
